In [1]:
import numpy as np
import itertools
from tqdm import tqdm

## 环境

In [2]:
"""
21点环境
=======
state: [玩家手牌列表], 庄家明牌
action: {叫牌(0)，停牌(1)}
reward: 胜利 1，失败 -1，平局 0
"""
import numpy as np


class BasePolicy:
    """策略基类
    """
    def act(self, obs):
        raise NotImplementedError('Policy.act Not Implemented')


class DealerPolicy(BasePolicy):
    """庄家策略

    手牌小于17要牌，否则停止
    """
    def act(self, obs):
        if obs < 17:
            return 0   #hit
        else:
            return 1   #stick


class BlackJack:
    def __init__(self):
        super().__init__()

        # 动作空间
        # 0: 要牌 hit
        # 1: 停止 stick
        self.action_space = (0, 1)

        # 游戏状态:
        # 0 玩家抽卡阶段，玩家停止抽卡时进入下一阶段
        # 1 庄家抽卡阶段
        # 2 结算阶段
        self.state = 0

        # 玩家与庄家的卡
        self.player_trajectory = []
        self.dealer_trajectory = []

        # 庄家策略
        self.dealer_policy = DealerPolicy()

    def reset(self):
        # 给每人发两张卡
        # 返回当前的观察（玩家的牌列表、庄家的明牌）
        self.player_trajectory = []
        self.player_trajectory.append(self._get_card())
        self.player_trajectory.append(self._get_card())
        self.dealer_trajectory = []
        self.dealer_trajectory.append(self._get_card())
        self.dealer_trajectory.append(self._get_card())

        self.state = 0

        return self._get_obs()

    def step(self, action):
        if action == 0:  # 玩家抽卡
            assert self.state == 0, '只能在 0 状态抽卡'

            # 抽卡
            self.player_trajectory.append(self._get_card())

            # 检测是否爆牌
            if self._is_blast(self.player_trajectory):
                return self._get_obs(), -1, True, {}

            return self._get_obs(), 0, False, {}
        
        elif action == 1: # 玩家停止要牌
            # 进入庄家决策阶段
            self.state += 1

            # 获取庄家观测
            dealer_obs = max_point(self.dealer_trajectory)
            # 庄家决策
            action = self.dealer_policy.act(dealer_obs)
            while action == 0:
                # 庄家抽排
                self.dealer_trajectory.append(self._get_card())
                # 计算当前点数之和
                dealer_obs = max_point(self.dealer_trajectory)
                # 爆牌检测，如果庄家爆牌，玩家得到1的回报
                if self._is_blast(self.dealer_trajectory):
                    return self._get_obs(), 1, True, {}
                # 如果没有爆牌，根据当前点数计算新的动作
                action = self.dealer_policy.act(dealer_obs)

            # 庄家停止要牌，开始结算
            self.state += 1
            player_point = max_point(self.player_trajectory)
            dealer_point = max_point(self.dealer_trajectory)
            # 比较点数，计算回报
            if player_point > dealer_point:
                reward = 1
            elif player_point == dealer_point:
                reward = 0
            else:
                reward = -1
            return self._get_obs(), reward, True, {}

        else:
            raise ValueError('非法动作')

    def _get_card(self):
        """抽卡

        Returns
        -------
        int
            抽到的卡(1-10)
        """
        card = np.random.randint(1, 14)
        card = min(card, 10)
        return card

    def _get_obs(self):
        """获取观测

        Returns
        -------
        Cards : list of int
            手牌列表
        Dealer's card 1 : int
            庄家的第一张牌
        """
        return (self.player_trajectory.copy(), self.dealer_trajectory[0])

    def _is_blast(self, traj):
        """检测是否爆牌

        Parameters
        ----------
        traj : list of int
            牌列表

        Returns
        -------
        blast : bool
            如果爆牌返回 True，否则 False
        """
        return max_point(traj) > 21


def max_point(traj):
    """工具函数，计算一个牌列表的最大点数

    Parameters
    ----------
    traj : list of int
        牌列表
    
    Returns
    -------
    point : int
        最大点数
    """
    s = 0
    num_ace = 0
    for card in traj:
        if card == 1:
            num_ace += 1
            s += 11
        else:
            s += card
    
    while s > 21 and num_ace > 0:
        s -= 10
        num_ace -= 1
    
    return s



## 小游戏

可跳过

In [3]:
l=['平局','你赢了','你输了']
env=BlackJack()
while(1):
    print('\n开始游戏！')
    env.reset()
    count=0
    while(1):
        count+=1
        print('round{}'.format(count))
        obs=env._get_obs()
        print('这是当前你的手牌:{}'.format(obs[0]),'\n','这是庄家的一张牌:{}'.format(obs[1]))

        while(1):
            action=int(input(r'怎么说？(0：要牌，1：停牌)'))
            if action!=0 and action!=1:
                print('打咩得斯哟！')
            else:
                break

        x=env.step(action)
        obs=x[0]
        if not x[2]:
            #玩家可以继续摸牌
            continue
        if env.state==0 and x[2]:
            #游戏是否在第一阶段结束
            print('----------')
            print('这是当前你的手牌:{}'.format(obs[0]),'\n','这是庄家的牌:{}'.format(env.dealer_trajectory))
            print('你的总和大于21，你输了')
            break
        elif env.state==1 and x[2]:
            #游戏是否在第二阶段结束
            print('----------')
            print('这是当前你的手牌:{}'.format(obs[0]),'\n','这是庄家的牌:{}'.format(env.dealer_trajectory))
            print('庄家的总和大于21，你赢了')
            break
        elif env.state==2 and x[2]:
            #游戏是否在第三阶段结束
            print('----------')
            print('这是当前你的手牌:{}'.format(obs[0]),'\n','这是庄家的牌:{}'.format(env.dealer_trajectory))
            reward=x[1]
            print(l[reward])
            break
    f=input('是否继续？')
    if not f:
        break


开始游戏！
round1
这是当前你的手牌:[10, 10] 
 这是庄家的一张牌:10
怎么说？(0：要牌，1：停牌)1
----------
这是当前你的手牌:[10, 10] 
 这是庄家的牌:[10, 3, 5]
你赢了
是否继续？


## 准备工作

根据书上的分析，该游戏有10\*2\*10=200个状态：

* 由于当手牌小于11时必要牌，当前手牌总和只可能为12-21，10种
* 当前手牌是否有可以作11用的A，2种可能
* 庄家的第一张牌，1-10两种可能

该游戏只有两个动作，hit和stick。

下面我们定义这些状态和动作。

In [24]:
player_sum_list=[12+i for i in range(10)]
if_usabelA=[0,1]
dealer_sum_list=[1+i for i in range(10)]

states_id_list=[-1+i for i in range(201)]   #-1代表terminal -1-199
states_list=list(itertools.product(if_usabelA,player_sum_list,dealer_sum_list))
states_dic={-1:'T'}
for i in range(200):
    states_dic[i]=states_list[i]

In [73]:
states_dic

{-1: 'T',
 0: (0, 12, 1),
 1: (0, 12, 2),
 2: (0, 12, 3),
 3: (0, 12, 4),
 4: (0, 12, 5),
 5: (0, 12, 6),
 6: (0, 12, 7),
 7: (0, 12, 8),
 8: (0, 12, 9),
 9: (0, 12, 10),
 10: (0, 13, 1),
 11: (0, 13, 2),
 12: (0, 13, 3),
 13: (0, 13, 4),
 14: (0, 13, 5),
 15: (0, 13, 6),
 16: (0, 13, 7),
 17: (0, 13, 8),
 18: (0, 13, 9),
 19: (0, 13, 10),
 20: (0, 14, 1),
 21: (0, 14, 2),
 22: (0, 14, 3),
 23: (0, 14, 4),
 24: (0, 14, 5),
 25: (0, 14, 6),
 26: (0, 14, 7),
 27: (0, 14, 8),
 28: (0, 14, 9),
 29: (0, 14, 10),
 30: (0, 15, 1),
 31: (0, 15, 2),
 32: (0, 15, 3),
 33: (0, 15, 4),
 34: (0, 15, 5),
 35: (0, 15, 6),
 36: (0, 15, 7),
 37: (0, 15, 8),
 38: (0, 15, 9),
 39: (0, 15, 10),
 40: (0, 16, 1),
 41: (0, 16, 2),
 42: (0, 16, 3),
 43: (0, 16, 4),
 44: (0, 16, 5),
 45: (0, 16, 6),
 46: (0, 16, 7),
 47: (0, 16, 8),
 48: (0, 16, 9),
 49: (0, 16, 10),
 50: (0, 17, 1),
 51: (0, 17, 2),
 52: (0, 17, 3),
 53: (0, 17, 4),
 54: (0, 17, 5),
 55: (0, 17, 6),
 56: (0, 17, 7),
 57: (0, 17, 8),
 58: (0, 

In [25]:
def GetStateID(state):
    #将state三元组转换成id
    #输入:三元组(if_usabelA,player_sum,dealer_card)
    #输出:id
    if state=='T':
        return -1
    else:
        #return (state[0]-12)*20+state[1]*10+(state[2]-1)
        return (state[0])*100+(state[1]-12)*10+(state[2]-1)
    
def GetState(obs):
    #从当前情况得到state三元组
    #input:obs(list,int)
    #output:三元组
    player_sum=max_point(obs[0])
    dealer_card=obs[1]
    if not 1 in obs[0]:
        if_usabelA=0
        return (if_usabelA,player_sum,dealer_card)
    else:
        if_usabelA= 0 if sum(obs[0])>=12 else 1   #当所有牌的总和大于等于12时，其中的A不能换为11；当总和小于等于11时，其中的一个A可以换为11
        return (if_usabelA,player_sum,dealer_card)
        

for i in states_dic.keys():
    assert GetStateID(states_dic[i])==i,'有问题'

## Policy

### 首先是辣鸡策略，作为Baseline。

In [56]:
class foolypolicy():
    def __init__(self):
        self.pi=np.zeros((200,2))
        #初始策略，只在20或21时停牌
        self.pi[80:100,1]=1
        self.pi[180:200,1]=1
        self.pi[0:80,0]=1
        self.pi[100:180,0]=1
    def act(self,obs):
        #根据观察给出动作
        s=GetStateID(GetState(obs))
        a=np.random.choice([0,1],p=self.pi[s])
        return a
    def test(self,num_episode,env):
        count=[0,0,0]   #平局、胜、负
        for i in tqdm(range(num_episode)):
            obs0=env.reset()
            a0=self.act(obs0)
            
            #Genetate episode from s0,a0
            while(1):
                x=env.step(a0)
                if not x[2]:
                    #游戏未结束
                    a0=self.act(x[0])
                else:
                    #游戏已结束
                    reward=x[1]
                    break
            
            #统计胜负
            count[reward]+=1
        
        print('在{}个对局中，胜{}，负{}，平{}，胜率为{}'.format(num_episode,count[1],count[-1],count[0],count[1]/num_episode))

In [57]:
fp=foolypolicy()
env=BlackJack()
fp.test(num_episode=100000,env=env)

100%|████████████████████████████████████████████████████████████████████████| 100000/100000 [00:26<00:00, 3812.87it/s]

在100000个对局中，胜28002，负67001，平4997，胜率为0.28002


### 然后是书上说的最优策略

In [86]:
class BestPolicy:
    
    def __init__(self):
        self.pi=np.zeros((200,2))
        
        #usable A
        for i in range(12,18):
            for j in range(1,11):
                if GetStateID((1,i,j))==99:
                    print(1,i,j)
                self.pi[GetStateID((1,i,j)),0]=1
        
        self.pi[GetStateID((1,18,1)),0]=1
        self.pi[GetStateID((1,18,2)),0]=1
        self.pi[GetStateID((1,18,9)),0]=1
        self.pi[GetStateID((1,18,10)),0]=1
        
        
        for i in range(19,22):
            for j in range(1,11):
                self.pi[GetStateID((1,i,j)),1]=1
        self.pi[GetStateID((1,18,3)),1]=1
        self.pi[GetStateID((1,18,4)),1]=1
        self.pi[GetStateID((1,18,5)),1]=1
        self.pi[GetStateID((1,18,6)),1]=1
        self.pi[GetStateID((1,18,7)),1]=1
        self.pi[GetStateID((1,18,8)),1]=1
        
        
        
        #no usable A
        for i in range(17,22):
            for j in range(1,11):
                self.pi[GetStateID((0,i,j)),1]=1
        for j in range(3,7):
            for i in range(13,17):
                self.pi[GetStateID((0,i,j)),1]=1
        self.pi[GetStateID((0,12,5)),1]=1
        self.pi[GetStateID((0,12,6)),1]=1
        #剩下的是hit
        for i in range(200):
            if not 1 in self.pi[i]:
                self.pi[i,0]=1
    
    def act(self,obs):
        #根据观察给出动作
        s=GetStateID(GetState(obs))
        a=np.random.choice([0,1],p=self.pi[s])
        return a
    def test(self,num_episode,env):
        count=[0,0,0]   #平局、胜、负
        for i in tqdm(range(num_episode)):
            obs0=env.reset()
            a0=self.act(obs0)
            
            #Genetate episode from s0,a0
            while(1):
                x=env.step(a0)
                if not x[2]:
                    #游戏未结束
                    a0=self.act(x[0])
                else:
                    #游戏已结束
                    reward=x[1]
                    break
            
            #统计胜负
            count[reward]+=1
        
        print('在{}个对局中，胜{}，负{}，平{}，胜率为{}'.format(num_episode,count[1],count[-1],count[0],count[1]/num_episode))

In [87]:
bp=BestPolicy()
env=BlackJack()
bp.test(num_episode=100000,env=env)

100%|████████████████████████████████████████████████████████████████████████| 100000/100000 [00:23<00:00, 4206.61it/s]

在100000个对局中，胜39436，负52555，平8009，胜率为0.39436


### 先使用MC_ES试一下

In [58]:
#由于开始状态完全随机，可以使用MC_ES
#在此问题中可以省去gamma和中间reward

class MC_ES:
    def __init__(self,num_state=200,num_action=2):
        self.Q=np.zeros((num_state,num_action))   #估计值
        self.count=np.ones((num_state,num_action))   #计数，用于增量学习
        self.num_state=num_state
        self.num_action=num_action
        self.pi=np.zeros((num_state,num_action))
        #初始策略，只在20或21时停牌
        self.pi[80:100,1]=1
        self.pi[180:200,1]=1
        self.pi[0:80,0]=1
        self.pi[100:180,0]=1
        
    def GenerateAction(self,s):
        #根据当前b策略生成动作
        #input:s(id)
        #print(s,self.pi[s])
        return np.random.choice([0,1],p=self.pi[s])
    
    def UpdatePiFromQ(self):
        tmp=np.argmax(self.Q,axis=1)
        self.pi=np.zeros((self.num_state,self.num_action))
        for i in range(self.num_state):
            self.pi[i][tmp[i]]=1
            
    
    def learn(self,num_episode,env):
        for i in tqdm(range(num_episode)):
            s_list=[]
            a_list=[]
            obs0=env.reset()
            s0=GetStateID(GetState(obs0))
            a0=self.GenerateAction(s0)
            s_list.append(s0)
            a_list.append(a0)
            
            #Genetate episode from s0,a0
            while(1):
                x=env.step(a0)
                if not x[2]:
                    #游戏未结束
                    s0=GetStateID(GetState(x[0]))
                    a0=self.GenerateAction(s0)
                    s_list.append(s0)
                    a_list.append(a0)
                else:
                    #游戏已结束
                    reward=x[1]
                    break
                    
            #update the estimate
            assert len(s_list)==len(a_list),'error!'
            l=len(s_list)
            for j in range(l):
                if not s_list[l-1-j] in s_list[:l-1-j]:
                    #first visit
                    s=s_list[l-1-j]
                    a=a_list[l-1-j]
                    self.Q[s][a]+=(reward-self.Q[s][a])/self.count[s][a]
                    self.count[s][a]+=1
            
            #update the policy
            #和书上不太一样，我是在一个episode之后更新而不是每一步更新一次,结果上应该没影响
            self.UpdatePiFromQ()
            
    def act(self,obs):
        #根据观察给出动作
        s=GetStateID(GetState(obs))
        a=np.random.choice([0,1],p=self.pi[s])
        return a
    
    def test(self,num_episode,env):
        count=[0,0,0]   #平局、胜、负
        for i in tqdm(range(num_episode)):
            obs0=env.reset()
            a0=self.act(obs0)
            
            #Genetate episode from s0,a0
            while(1):
                x=env.step(a0)
                if not x[2]:
                    #游戏未结束
                    a0=self.act(x[0])
                else:
                    #游戏已结束
                    reward=x[1]
                    break
            
            #统计胜负
            count[reward]+=1
        
        print('在{}个对局中，胜{}，负{}，平{}，胜率为{}'.format(num_episode,count[1],count[-1],count[0],count[1]/num_episode))

In [59]:
mc=MC_ES()
env=BlackJack()
mc.learn(num_episode=100000,env=env)
mc.test(num_episode=100000,env=env)

100%|████████████████████████████████████████████████████████████████████████| 100000/100000 [00:26<00:00, 3794.75it/s]

在100000个对局中，胜33041，负59472，平7487，胜率为0.33041


In [46]:
for i in range(200):
    print(states_dic[i],mc.pi[i])

(0, 12, 1) [1. 0.]
(0, 12, 2) [0. 1.]
(0, 12, 3) [1. 0.]
(0, 12, 4) [0. 1.]
(0, 12, 5) [1. 0.]
(0, 12, 6) [1. 0.]
(0, 12, 7) [1. 0.]
(0, 12, 8) [0. 1.]
(0, 12, 9) [1. 0.]
(0, 12, 10) [1. 0.]
(0, 13, 1) [1. 0.]
(0, 13, 2) [1. 0.]
(0, 13, 3) [1. 0.]
(0, 13, 4) [0. 1.]
(0, 13, 5) [0. 1.]
(0, 13, 6) [1. 0.]
(0, 13, 7) [1. 0.]
(0, 13, 8) [1. 0.]
(0, 13, 9) [0. 1.]
(0, 13, 10) [1. 0.]
(0, 14, 1) [1. 0.]
(0, 14, 2) [1. 0.]
(0, 14, 3) [1. 0.]
(0, 14, 4) [0. 1.]
(0, 14, 5) [1. 0.]
(0, 14, 6) [1. 0.]
(0, 14, 7) [1. 0.]
(0, 14, 8) [1. 0.]
(0, 14, 9) [0. 1.]
(0, 14, 10) [1. 0.]
(0, 15, 1) [1. 0.]
(0, 15, 2) [1. 0.]
(0, 15, 3) [0. 1.]
(0, 15, 4) [0. 1.]
(0, 15, 5) [1. 0.]
(0, 15, 6) [1. 0.]
(0, 15, 7) [0. 1.]
(0, 15, 8) [1. 0.]
(0, 15, 9) [1. 0.]
(0, 15, 10) [1. 0.]
(0, 16, 1) [1. 0.]
(0, 16, 2) [1. 0.]
(0, 16, 3) [1. 0.]
(0, 16, 4) [1. 0.]
(0, 16, 5) [1. 0.]
(0, 16, 6) [0. 1.]
(0, 16, 7) [0. 1.]
(0, 16, 8) [1. 0.]
(0, 16, 9) [1. 0.]
(0, 16, 10) [1. 0.]
(0, 17, 1) [0. 1.]
(0, 17, 2) [0. 1.]
(0, 17,

### n-step Sarsa

In [92]:
class NStepSarsa:
    
    def __init__(self,gamma=1,alpha=0.1,n=3,num_state=200,num_action=2,off_policy=True,epsilon=None):
        super().__init__()
        self.gamma=gamma
        self.alpha=alpha
        self.n=n
        self.num_state=num_state
        self.num_action=num_action   #实际上严格不应该这么写，因为每个state对应的action不一定一样，在这个问题中我们可以这么写，后面也有很多这种不太严格的地方
        self.off_policy=off_policy
        
        self.Q=np.zeros((num_state,num_action))   #估计
        self.pi=np.zeros((num_state,num_action))   #pi策略

        if self.off_policy:
            self.pi[:,0]=1   #初始化pi策略
            self.b=np.full((num_state,num_action),0.5)   #默认的b策略
        else:
            self.epsilon=epsilon
            self.pi[:,0]=1-self.epsilon
            self.pi[:,1]=self.epsilon  #初始化pi策略
            self.b=self.pi   #只在产生动作中用到，on-policy我会直接将ro设为1
        
    
    def UpdatePiFromQ(self):
        #根据估计值得到当前的最优策略(off-policy:determinal,on-policy:epsilon-greedy)
        tmp=np.argmax(self.Q,axis=1)
        if self.off_policy:
            self.pi=np.zeros((self.num_state,self.num_action))
            for i in range(self.num_state):
                self.pi[i][tmp[i]]=1
        else:
            self.pi=np.full((self.num_state,self.num_action),self.epsilon/(self.num_action-1))
            for i in range(self.num_state):
                self.pi[i][tmp[i]]=1-self.epsilon
            self.b=self.pi
    
    def GenerateAction(self,s):
        #根据当前b策略生成动作
        #input:s(id)
        return np.random.choice([0,1],p=self.b[s])
    
    def learn(self,num_episode,env):
        #估计
        for i in tqdm(range(num_episode)):
            obs0=env.reset()
            s_list=[]
            a_list=[]
            reward=[]   #注意reward列表的序号！跟书上有区别！
            #s0=np.random.randint(0,200)   #初始状态
            s0=GetStateID(GetState(obs0))   #初始状态
            s_list.append(s0)
            a0=self.GenerateAction(s0)   #初始动作
            a_list.append(a0)
            T=np.inf
            t=0
            
            while(1):
                
                if t<T:
                    #take action A
                    x=env.step(a_list[t])
                    if x[2]:
                        #一轮游戏结束
                        final_reward=x[1]   #最终时刻的奖励
                        reward.append(0)
                        reward.append(final_reward)
                        s_list.append(-1)
                        T=t+1
                    else:
                        #尚未结束
                        obs=x[0]
                        s=GetStateID(GetState(obs))
                        s_list.append(s)
                        reward.append(0)
                        a=self.GenerateAction(s)
                        a_list.append(a)
                tao=t-self.n+1   #当前需要更新的位置
                
                if tao>=0:
                    #需要更新
                    ro=1
                    if self.off_policy:
                        for i in range(tao+1,min(tao+self.n,T-1)):
                            s=s_list[i]
                            a=a_list[i]
                            ro*=self.pi[s][a]/self.b[s][a]
                    G=0 if tao+self.n<T else reward[-1]   #书上公式在这个特定问题上的简化
                    if tao+self.n<T:
                        G+=(self.gamma**self.n)*self.Q[s_list[tao+self.n]][a_list[tao+self.n]]
                    self.Q[s_list[tao]][a_list[tao]]=self.Q[s_list[tao]][a_list[tao]]+self.alpha*ro*(G-self.Q[s_list[tao]][a_list[tao]])
                    self.UpdatePiFromQ()
                t+=1
                if tao==T-1:
                    break
                    
    def act(self,obs):
        #根据观察给出动作
        s=GetStateID(GetState(obs))
        a=np.random.choice([0,1],p=self.pi[s])
        return a
    def test(self,num_episode,env):
        count=[0,0,0]   #平局、胜、负
        for i in tqdm(range(num_episode)):
            obs0=env.reset()
            a0=self.act(obs0)
            
            #Genetate episode from s0,a0
            while(1):
                x=env.step(a0)
                if not x[2]:
                    #游戏未结束
                    a0=self.act(x[0])
                else:
                    #游戏已结束
                    reward=x[1]
                    break
            
            #统计胜负
            count[reward]+=1
        
        print('在{}个对局中，胜{}，负{}，平{}，胜率为{}'.format(num_episode,count[1],count[-1],count[0],count[1]/num_episode))
                    

In [93]:
ss=NStepSarsa(n=30)
env=BlackJack()
ss.learn(num_episode=100000,env=env)
ss.test(num_episode=100000,env=env)

100%|████████████████████████████████████████████████████████████████████████| 100000/100000 [00:24<00:00, 4153.55it/s]

在100000个对局中，胜38573，负55444，平5983，胜率为0.38573


In [95]:
for i in range(200):
    print(states_dic[i],ss.pi[i])

(0, 12, 1) [0. 1.]
(0, 12, 2) [0. 1.]
(0, 12, 3) [0. 1.]
(0, 12, 4) [0. 1.]
(0, 12, 5) [0. 1.]
(0, 12, 6) [0. 1.]
(0, 12, 7) [1. 0.]
(0, 12, 8) [0. 1.]
(0, 12, 9) [1. 0.]
(0, 12, 10) [0. 1.]
(0, 13, 1) [0. 1.]
(0, 13, 2) [0. 1.]
(0, 13, 3) [0. 1.]
(0, 13, 4) [0. 1.]
(0, 13, 5) [0. 1.]
(0, 13, 6) [0. 1.]
(0, 13, 7) [1. 0.]
(0, 13, 8) [1. 0.]
(0, 13, 9) [1. 0.]
(0, 13, 10) [0. 1.]
(0, 14, 1) [0. 1.]
(0, 14, 2) [0. 1.]
(0, 14, 3) [1. 0.]
(0, 14, 4) [0. 1.]
(0, 14, 5) [1. 0.]
(0, 14, 6) [0. 1.]
(0, 14, 7) [1. 0.]
(0, 14, 8) [0. 1.]
(0, 14, 9) [1. 0.]
(0, 14, 10) [0. 1.]
(0, 15, 1) [1. 0.]
(0, 15, 2) [0. 1.]
(0, 15, 3) [0. 1.]
(0, 15, 4) [0. 1.]
(0, 15, 5) [0. 1.]
(0, 15, 6) [0. 1.]
(0, 15, 7) [0. 1.]
(0, 15, 8) [0. 1.]
(0, 15, 9) [0. 1.]
(0, 15, 10) [0. 1.]
(0, 16, 1) [1. 0.]
(0, 16, 2) [0. 1.]
(0, 16, 3) [0. 1.]
(0, 16, 4) [0. 1.]
(0, 16, 5) [0. 1.]
(0, 16, 6) [0. 1.]
(0, 16, 7) [0. 1.]
(0, 16, 8) [0. 1.]
(0, 16, 9) [0. 1.]
(0, 16, 10) [0. 1.]
(0, 17, 1) [0. 1.]
(0, 17, 2) [0. 1.]
(0, 17,